<a href="https://colab.research.google.com/github/AlexWhite-USDA/NDII_33km/blob/main/NDII_33km.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Alex White

April 22, 2021

Compute Landsat-derived Normalized Difference Infrared Index (NDII, aka NDWI for "water index" in this script) at input point within incrementally-increasing circular regions from 1- to 33-km diameter, and view results over user-selected date range. View NAIP availability and time series at input point.

*Updated to include water mask.

*2024 updates:

*   Project ID in ee.Initialize
*   Collection IDs per https://developers.google.com/earth-engine/landsat_c1_to_c2
*   Cloud and shadow mask expression (now same for all collections)
*   Band names, e.g. B4 to SR_B4, and pixel_qa to QA_PIXEL
*   Image Properties, e.g. SATELLITE to SPACECRAFT_ID
*   Get image date from LANDSAT_PRODUCT_ID
*   Hansen global change dataset from UMD_hansen_global_forest_change_2015 to UMD_hansen_global_forest_change_2023_v1_11
*   Add Landsat 9

In [1]:
#@title Initialization
# Import libraries and initialize Earth Engine
import ee
import folium
import pandas as pd
import altair as alt
from folium import plugins
import ipywidgets as widgets
from IPython.display import display
from google.colab import files

ee.Authenticate()
ee.Initialize(project='coastal-campus-426814-d1')

In [133]:
#@title User input { display-mode: "form", run: "auto" }
#@markdown ### Enter study area name
alias = 'Choptank_CR04' #@param {type:'string'}

#@markdown ### Enter coordinates
lat = 39.02605 #@param {type:'number'}
lon = -76.13078 #@param {type:'number'}

#@markdown ### Enter date range
start = '2011-04-01' #@param {type:'date'}
end = '2024-04-01' #@param {type:'date'}

#@markdown ### Enter threshold value (percent) for minimum acceptable coverage of a 1-km diameter circle surrounding point, after cloud masking
pct_cov = 70 #@param {type:"slider", min:0, max:100, step:1}

#@markdown ### Run remaining code cells (Ctrl+F10) to view results.

# Print user input
coordStr = '(' + str(lat) + ', ' + str(lon) + ')'
print('alias: ' + alias)
print('coordinates: ' + coordStr)
print('date range: ' + start + ' - ' + end)
print('1-km study area coverage after cloud-masking: ≥' + str(pct_cov) + '%')

alias: Choptank_CR04
coordinates: (39.02605, -76.13078)
date range: 2011-04-01 - 2024-04-01
1-km study area coverage after cloud-masking: ≥70%


In [125]:
#@title Define functions
# Cloud mask function, updated for Collection 2
def cloudMask(scene):
  qa_mask = scene.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
  return scene.updateMask(qa_mask)

# NDWI function for Landsats 5 and 7
def ndwiL57(scene):
  ndwi = scene.normalizedDifference(['SR_B4', 'SR_B5']).select([0], ['NDWI'])
  return scene.addBands(ndwi)

# NDWI function for Landsats 8 and 9
def ndwiL89(scene):
  ndwi = scene.normalizedDifference(['SR_B5', 'SR_B6']).select([0], ['NDWI'])
  return scene.addBands(ndwi)

# Count pixels in buffered area of interest (AOI)
def aoiCount(scene):
  c = ee.Number((scene.reduceRegion(ee.Reducer.count(), aoi).get('NDWI')))
  return scene.set({'aoi_pixel_count': c})

# Average NDWI in buffered AOI
def aoiNDWI(scene):
  m = ee.Number((scene.reduceRegion(ee.Reducer.mean(), aoi).get('NDWI')))
  return scene.set({'aoi_NDWI_mean': m})

# Filter collection by user-input AOI percent-coverage
def filterAOI(collection):
  apc_max = collection.aggregate_max('aoi_pixel_count').getInfo()
  apc_set = apc_max * pct_cov / 100
  return collection.filter(ee.Filter.gte('aoi_pixel_count', apc_set))

# Define a method for displaying Earth Engine image tiles on a folium map
# https://colab.research.google.com/github/giswqs/qgis-earthengine-examples/blob/master/Folium/ee-api-folium-setup.ipynb
def add_ee_layer(self, ee_object, vis_params, name):
  # display ee.Image()
  if isinstance(ee_object, ee.image.Image):
      map_id_dict = ee.Image(ee_object).getMapId(vis_params)
      folium.raster_layers.TileLayer(
      tiles = map_id_dict['tile_fetcher'].url_format,
      attr = 'Google Earth Engine',
      name = name,
      overlay = True,
      control = True
      ).add_to(self)
  # display ee.ImageCollection()
  elif isinstance(ee_object, ee.imagecollection.ImageCollection):
      ee_object_new = ee_object.mosaic()
      map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
      folium.raster_layers.TileLayer(
      tiles = map_id_dict['tile_fetcher'].url_format,
      attr = 'Google Earth Engine',
      name = name,
      overlay = True,
      control = True
      ).add_to(self)
  # display ee.Geometry()
  elif isinstance(ee_object, ee.geometry.Geometry):
      folium.GeoJson(
      data = ee_object.getInfo(),
      name = name,
      overlay = True,
      control = True
  ).add_to(self)
  # display ee.FeatureCollection()
  elif isinstance(ee_object, ee.featurecollection.FeatureCollection):
      ee_object_new = ee.Image().paint(ee_object, 0, 2)
      map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
      folium.raster_layers.TileLayer(
      tiles = map_id_dict['tile_fetcher'].url_format,
      attr = 'Google Earth Engine',
      name = name,
      overlay = True,
      control = True
  ).add_to(self)

# Add EE drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

# Get image collection info, parse JSON, and create DataFrame
def parseInfo(collection,properties):
  i = collection.getInfo()
  f = i['features']
  bigList = []
  for feature in f:
    l = []
    for p in properties:
      try:
        l.append(feature['properties'][p])
      except:
        l.append(float('NaN'))
    bigList.append(l)
  return pd.DataFrame(bigList, columns = properties)

# Add image statistics to buffer features
def addStats(scene):
  meansFeatures = scene.reduceRegions(buffers, ee.Reducer.mean())
  stdevsFeatures = scene.reduceRegions(meansFeatures, ee.Reducer.stdDev())
  countsFeatures = scene.reduceRegions(stdevsFeatures, ee.Reducer.count())
  return countsFeatures

# Average reflectance values in NAIP AOI
def aoiAvg(scene):
  r = ee.Number((scene.reduceRegion(ee.Reducer.mean(), naip_aoi).get('R')))
  g = ee.Number((scene.reduceRegion(ee.Reducer.mean(), naip_aoi).get('G')))
  b = ee.Number((scene.reduceRegion(ee.Reducer.mean(), naip_aoi).get('B')))
  return scene.set({'aoi_red': r,'aoi_green': g,'aoi_blue': b})

# Mask water pixels
# https://developers.google.com/earth-engine/tutorials/tutorial_api_05
def waterMask(scene):
  # Load or import the Hansen et al. forest change dataset
  hansenImage = ee.Image('UMD/hansen/global_forest_change_2023_v1_11')

  # Select the land/water mask
  datamask = hansenImage.select('datamask')

  # Create a binary mask
  mask = datamask.eq(1)

  # Update the composite mask with the water mask
  return scene.updateMask(mask)

In [134]:
#@title Run server-side analysis
# Set centroid for area of interest (AOI)
point = ee.Geometry.Point([lon, lat])

# Set minimum AOI for Landsat (1-km diameter)
aoi = point.buffer(500)

# Set AOI for NAIP (10-m diameter)
naip_aoi = point.buffer(5)

# Create collection of buffer features for Landsat
bs = []
for b in range(1, 34):
  b_name = str(b) + '_km'
  bs.append(ee.Feature(point.buffer(b*500), {'name':b_name}))
buffers = ee.FeatureCollection(bs)

# Load reflectance data, mask water, mask clouds, compute NDWI, and count pixels
l5sr = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
.filterDate(start, end) \
.filterBounds(aoi) \
.map(waterMask) \
.map(cloudMask) \
.map(ndwiL57) \
.map(aoiCount) \
.map(aoiNDWI)

l7sr = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
.filterDate(start, end) \
.filterBounds(aoi) \
.map(waterMask) \
.map(cloudMask) \
.map(ndwiL57) \
.map(aoiCount) \
.map(aoiNDWI)

l8sr = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
.filterDate(start, end) \
.filterBounds(aoi) \
.map(waterMask) \
.map(cloudMask) \
.map(ndwiL89) \
.map(aoiCount) \
.map(aoiNDWI)

l9sr = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') \
.filterDate(start, end) \
.filterBounds(aoi) \
.map(waterMask) \
.map(cloudMask) \
.map(ndwiL89) \
.map(aoiCount) \
.map(aoiNDWI)

# Merge Landsat collections
merged = l5sr.select('NDWI').merge(l7sr.select('NDWI')) \
.merge(l8sr.select('NDWI')).merge(l9sr.select('NDWI'))

# Filter Landsat collection by AOI pixel count
ndwi_all = filterAOI(merged)

# Load NAIP and average band values in AOI
naip = ee.ImageCollection('USDA/NAIP/DOQQ') \
.filterDate(start, end) \
.filterBounds(naip_aoi) \
.map(aoiAvg)

In [135]:
#@title Run client-side analysis
# Select image properties to extract from Landsat collection
properties = ('SPACECRAFT_ID','LANDSAT_PRODUCT_ID','system:time_start','aoi_pixel_count','aoi_NDWI_mean')

# Select image properties to extract from NAIP collection
props = ['system:index','system:time_start','system:time_end','aoi_blue','aoi_green','aoi_red']

# Parse JSON info and create DataFrames
infoDF = parseInfo(ndwi_all,properties)
infoDF['Date'] = infoDF.apply(lambda x: pd.to_datetime(x['LANDSAT_PRODUCT_ID'][17:25], format='%Y%m%d'), axis=1)
naipDF = parseInfo(naip,props).sort_values('system:time_start')

# Convert NAIP epoch timestamps and drop NaNs
start_dt = pd.to_datetime(naipDF['system:time_start'], unit='ms').rename('start_date')
end_dt = pd.to_datetime(naipDF['system:time_end'], unit='ms').rename('end_date')
joinDF = naipDF.join(start_dt).join(end_dt)
naipDF = joinDF.dropna()

# Print number of records in DataFrames
print(str(len(infoDF.index)) + ' images in Landsat collection.')
print(str(len(naipDF.index)) + ' images in NAIP collection.')

361 images in Landsat collection.
6 images in NAIP collection.


In [136]:
infoDF

,SPACECRAFT_ID,LANDSAT_PRODUCT_ID,system:time_start,aoi_pixel_count,aoi_NDWI_mean,Date
0,LANDSAT_5,LT05_L2SP_014033_20110425_20200822_02_T1,1303745395991,866,0.086246,2011-04-25
1,LANDSAT_5,LT05_L2SP_014033_20110511_20200822_02_T1,1305127791364,647,0.008769,2011-05-11
2,LANDSAT_5,LT05_L2SP_014033_20110527_20200822_02_T1,1306510186521,859,0.023345,2011-05-27
3,LANDSAT_5,LT05_L2SP_014033_20110612_20200822_02_T1,1307892577764,807,0.092513,2011-06-12
4,LANDSAT_5,LT05_L2SP_014033_20110628_20200822_02_T1,1309274970982,866,0.222561,2011-06-28
...,...,...,...,...,...,...
356,LANDSAT_9,LC09_L2SP_015033_20231103_20231104_02_T1,1699026394403,866,-0.015768,2023-11-03
357,LANDSAT_9,LC09_L2SP_015033_20231119_20231120_02_T1,1700408801132,866,0.000817,2023-11-19
358,LANDSAT_9,LC09_L2SP_015033_20231221_20231223_02_T1,1703173607594,865,0.039252,2023-12-21
359,LANDSAT_9,LC09_L2SP_015033_20240122_20240124_02_T1,1705938400453,865,0.562368,2024-01-22


In [137]:
#@title Plot NAIP RGB time series
b_points = alt.Chart(
    naipDF,
    height=400,
    width=1000,
    title='NAIP DNs in 10-m Area of Interest'
    ).mark_circle(
        size=60,
        opacity=0.8,
        color='blue'
        ).encode(
            x='start_date',
            y='aoi_blue',
            tooltip=['system:index','start_date','end_date','aoi_blue']
            ).interactive()

b_lines = alt.Chart(naipDF).mark_line(color='blue',strokeWidth=1).encode(
  x='start_date',
  y='aoi_blue'
)

g_points = alt.Chart(naipDF).mark_circle(
        size=60,
        opacity=0.8,
        color='green'
        ).encode(
            x='start_date',
            y='aoi_green',
            tooltip=['system:index','start_date','end_date','aoi_green']
            ).interactive()

g_lines = alt.Chart(naipDF).mark_line(color='green',strokeWidth=1).encode(
  x='start_date',
  y='aoi_green'
)

r_points = alt.Chart(naipDF).mark_circle(
        size=60,
        opacity=0.8,
        color='red'
        ).encode(
            x='start_date',
            y='aoi_red',
            tooltip=['system:index','start_date','end_date','aoi_red']
            ).interactive()

r_lines = alt.Chart(naipDF).mark_line(color='red',strokeWidth=1).encode(
  x='start_date',
  y='aoi_red'
)

b_lines + b_points + g_lines + g_points + r_lines + r_points

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

In [138]:
#@title Plot NDWI time series
points = alt.Chart(
    infoDF,
    height=400,
    width=1000,
    title='NDWI in 1-km Area of Interest'
    ).mark_circle(
        size=60,
        opacity=0.8
        ).encode(
            x='Date',
            y='aoi_NDWI_mean',
            color='SPACECRAFT_ID',
            tooltip=['Date','SPACECRAFT_ID','aoi_pixel_count','aoi_NDWI_mean']
            ).interactive()

lines = alt.Chart(infoDF).mark_line(color='gray',strokeWidth=1).encode(
  x='Date',
  y='aoi_NDWI_mean'
)

lines + points

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

In [141]:
#@title Enter NDWI image date and study area diameter (in km), to view on map. { display-mode: "form", run: "auto" }
date_input = '2023-07-07' #@param {type:"date"}
buffer_size = 1 #@param {type:"slider", min:1, max:33, step:1}

try:
  # Find image(s) for user-selected date
  map_df = infoDF[infoDF['Date'] == date_input]
except:
  print('No image found for date_input = ' + date_input + '.')
  print('Please check date and try again.')

# Create a folium map object
m = folium.Map(location=[lat, lon], zoom_start=11)

# Add NAIP imagery to the map, as basemap
m.add_ee_layer(ee.ImageCollection('USDA/NAIP/DOQQ')
            .filter(ee.Filter.date('2016-01-01', '2020-12-31'))
            .select(['R', 'G', 'B']),{'min': 0.0, 'max': 255.0},'NAIP basemap')

# Add NAIP imagery to the map, per scene
for r in naipDF.index:
  i = naipDF.at[r,'system:index']
  s = naipDF.at[r,'system:time_start']
  e = naipDF.at[r,'system:time_end']
  n = 'NAIP ' + str(naipDF.at[r,'start_date']).split(' ')[0] + ' ' + i

  m.add_ee_layer(ee.Image('USDA/NAIP/DOQQ/' + i)
            .select(['R', 'G', 'B']),{'min': 0.0, 'max': 255.0},n)

# Add buffer polygon to the map
folium.Circle(
    radius=buffer_size * 500,
    location=[lat, lon],
    popup='buffer',
    color='crimson',
    fill=False
).add_to(m)

# Add point to the map
folium.Marker(
    [lat, lon], popup=coordStr
).add_to(m)

# Set visualization parameters
# https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_8DAY_NDWI
vis_params = {
  'min': 0,
  'max': 1.0,
  'palette': ['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff']
}

# Map image(s) and list metadata key-value pairs
img_list = []
for record in range(len(map_df.index)):
  map_epoch = map_df.iloc[record]['system:time_start']
  map_date = str(map_df.iloc[record]['Date'])
  map_sat = map_df.iloc[record]['SPACECRAFT_ID']
  map_id = map_df.iloc[record]['LANDSAT_PRODUCT_ID']
  map_name = str(record+1) + ' ' + map_sat + ' ' + map_date
  img_list.append((map_name,[map_epoch,map_date,map_sat,map_id]))

  # Add image to the map
  img = ndwi_all.filterDate(int(map_epoch)).first()
  m.add_ee_layer(img, vis_params, map_name)

# Add a layer control panel to the map
m.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(m)

# Display the map
display(m)

# Show dropdown list of images
w = widgets.Dropdown(
    options=img_list,
    description='Image:',
    disabled=False,
)
display(w)

# Show button to run stats for selected image
print('\nClick button to view and download NDWI statistics for selected image:')
button = widgets.Button(description='Get Stats')
output = widgets.Output()

def on_button_clicked(b):
  with output:
    # Generate feature collection for user-selected image
    bufStats = addStats(ndwi_all.filterDate(int(w.value[0])).first())

    # Parse JSON info and create DataFrame
    fc = bufStats.getInfo()
    c = fc['features']
    bigList2 = []

    for feature in c:
      l = []
      for p in feature['properties']:
        l.append(feature['properties'][p])
      bigList2.append(l)

    bufDF = pd.DataFrame(bigList2,
                        columns = ['count','mean','diameter','stDev']
                        ).set_index('diameter')

    # Prepare output
    header = 'NDWI Statistics in Circular Regions at '  + alias + '\n' \
      + 'center point: ' + coordStr + '\n' \
      + 'spacecraft: ' + w.value[2] + '\n' \
      + 'sensing time: ' + w.value[1] + '\n' \
      + 'Landsat ID: ' + w.value[3] + '\n'

  # Write output and invoke browser download
  filename = w.value[3] + '_33km_NDWI.txt'
  with open(filename, 'w') as f:
    f.write(header + '\n')
    f.write(bufDF.to_csv())
    files.download(filename)

  print(header)
  print(bufDF,'\n\n')

button.on_click(on_button_clicked)
display(button, output)

Dropdown(description='Image:', options=(('1 LANDSAT_9 2023-07-07 00:00:00', [1688744389556, '2023-07-07 00:00:…


Click button to view and download NDWI statistics for selected image:


Button(description='Get Stats', style=ButtonStyle())

Output()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

NDWI Statistics in Circular Regions at Choptank_CR04
center point: (39.02605, -76.13078)
spacecraft: LANDSAT_9
sensing time: 2023-07-07 00:00:00
Landsat ID: LC09_L2SP_014033_20230707_20230711_02_T1

           count      mean     stDev
diameter                            
1_km         685  0.268763  0.079932
2_km        2312  0.236017  0.111447
3_km        5666  0.195139  0.136403
4_km        9964  0.191577  0.129677
5_km       15965  0.188561  0.120857
6_km       22998  0.186127  0.116373
7_km       29781  0.183620  0.113035
8_km       36739  0.180124  0.111749
9_km       44262  0.178680  0.112455
10_km      52793  0.178663  0.113725
11_km      60459  0.179817  0.112777
12_km      66586  0.179813  0.112302
13_km      72556  0.179658  0.112208
14_km      82327  0.181200  0.110874
15_km      93425  0.181195  0.109688
16_km     106131  0.181086  0.109227
17_km     118441  0.180792  0.110256
18_km     133509  0.180249  0.110624
19_km     150547  0.181735  0.109345
20_km     166193  0.1827